# 51-并发编程模式

## 📚 用途说明

**学习目标**：
- 掌握生产者消费者模式的异步实现
- 学会工作池和线程池的并发管理
- 理解扇出扇入模式的数据处理流程
- 能够应用并发模式解决实际问题

**前置要求**：
- 已完成异步Web框架学习
- 熟练掌握协程和事件循环机制
- 了解基本的并发编程概念
- 理解队列和同步原语的使用

**与LangChain关联**：
- 生产者消费者模式优化LangChain的数据处理流程
- 工作池模式提升LangChain的并发推理能力
- 扇出扇入模式增强LangChain的并行处理效率
- 并发模式为LangChain的分布式部署提供基础

---

## 🔢 知识点覆盖

### 6.6 并发编程模式 [⭐⭐进阶]
**知识点说明**：并发编程模式是构建高性能异步系统的核心技术，对于LangChain的大规模数据处理和并行推理至关重要。

**学习要求**：
- 掌握生产者消费者模式的异步实现
- 学会工作池和线程池的并发管理
- 理解扇出扇入模式的数据处理流程
- 能够应用并发模式解决实际问题

**案例要求**：
- 实现完整的并发模式系统
- 进行并发性能对比和分析
- 应用并发模式解决实际问题
- 验证点：能独立设计和实现并发解决方案

In [ ]:
print("🔄 并发编程模式:")
print("=" * 50)

# 导入必要的库
import asyncio
import time
import datetime
import json
import random
import uuid
import threading
from typing import List, Dict, Any, Optional, AsyncGenerator, Callable
from dataclasses import dataclass, asdict
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from collections import deque
import weakref

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 生产者消费者模式
print(f"📝 1. 生产者消费者模式:")

# 1.1 异步生产者消费者基础实现
print(f"\n   🔍 1.1 异步生产者消费者基础实现:")

@dataclass
class TaskItem:
    """任务项"""
    id: str
    data: Any
    priority: int = 0
    created_at: float = None
    
    def __post_init__(self):
        if self.created_at is None:
            self.created_at = time.time()

@dataclass
class ResultItem:
    """结果项"""
    task_id: str
    result: Any
    processing_time: float
    worker_id: str
    completed_at: float = None
    
    def __post_init__(self):
        if self.completed_at is None:
            self.completed_at = time.time()

class AsyncProducerConsumer:
    """异步生产者消费者模式实现"""
    
    def __init__(self, max_queue_size: int = 100, max_workers: int = 3):
        self.task_queue = asyncio.Queue(maxsize=max_queue_size)
        self.result_queue = asyncio.Queue(maxsize=max_queue_size)
        self.max_workers = max_workers
        self.workers = []
        self.running = False
        self.stats = {
            "produced_tasks": 0,
            "consumed_tasks": 0,
            "failed_tasks": 0,
            "total_processing_time": 0
        }
    
    async def start(self):
        """启动生产者消费者系统"""
        self.running = True
        
        # 启动消费者工作进程
        self.workers = [
            asyncio.create_task(self._worker(f"worker-{i}"))
            for i in range(self.max_workers)
        ]
        
        print(f"      🚀 生产者消费者系统启动 ({self.max_workers} 个工作进程)")
    
    async def stop(self):
        """停止生产者消费者系统"""
        self.running = False
        
        # 等待所有工作进程完成
        await asyncio.gather(*self.workers, return_exceptions=True)
        
        print(f"      🛑 生产者消费者系统停止")
    
    async def produce(self, task: TaskItem):
        """生产任务"""
        try:
            await self.task_queue.put(task)
            self.stats["produced_tasks"] += 1
            return True
        except asyncio.QueueFull:
            print(f"      ⚠️ 任务队列已满，丢弃任务: {task.id}")
            return False
    
    async def consume(self) -> Optional[ResultItem]:
        """消费结果"""
        try:
            result = await asyncio.wait_for(
                self.result_queue.get(), 
                timeout=1.0
            )
            return result
        except asyncio.TimeoutError:
            return None
    
    async def _worker(self, worker_id: str):
        """工作进程"""
        print(f"      👷 {worker_id} 开始工作")
        
        while self.running:
            try:
                # 获取任务（超时1秒）
                task = await asyncio.wait_for(
                    self.task_queue.get(), 
                    timeout=1.0
                )
                
                # 处理任务
                start_time = time.time()
                result = await self._process_task(task, worker_id)
                processing_time = time.time() - start_time
                
                # 创建结果项
                result_item = ResultItem(
                    task_id=task.id,
                    result=result,
                    processing_time=processing_time,
                    worker_id=worker_id
                )
                
                # 发送结果
                await self.result_queue.put(result_item)
                
                # 更新统计
                self.stats["consumed_tasks"] += 1
                self.stats["total_processing_time"] += processing_time
                
                print(f"      ✅ {worker_id} 完成任务: {task.id} ({processing_time:.3f}s)")
                
            except asyncio.TimeoutError:
                continue  # 没有任务，继续等待
            except Exception as e:
                self.stats["failed_tasks"] += 1
                print(f"      ❌ {worker_id} 任务失败: {e}")
        
        print(f"      👷 {worker_id} 停止工作")
    
    async def _process_task(self, task: TaskItem, worker_id: str) -> Any:
        """处理任务（可重写）"""
        # 模拟任务处理
        processing_time = random.uniform(0.1, 1.0)
        await asyncio.sleep(processing_time)
        
        # 模拟处理结果
        if isinstance(task.data, dict):
            return {
                **task.data,
                "processed_by": worker_id,
                "processing_time": processing_time,
                "timestamp": time.time()
            }
        else:
            return f"Processed: {task.data} by {worker_id}"
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        stats = self.stats.copy()
        
        if stats["consumed_tasks"] > 0:
            stats["average_processing_time"] = (
                stats["total_processing_time"] / stats["consumed_tasks"]
            )
        else:
            stats["average_processing_time"] = 0
        
        stats["queue_size"] = self.task_queue.qsize()
        stats["result_queue_size"] = self.result_queue.qsize()
        
        return stats

# 1.2 优先级生产者消费者
print(f"\n   🎯 1.2 优先级生产者消费者:")

class PriorityProducerConsumer:
    """优先级生产者消费者模式"""
    
    def __init__(self, max_queue_size: int = 100, max_workers: int = 3):
        self.task_queue = asyncio.PriorityQueue(maxsize=max_queue_size)
        self.result_queue = asyncio.Queue(maxsize=max_queue_size)
        self.max_workers = max_workers
        self.workers = []
        self.running = False
        self.stats = {
            "high_priority_tasks": 0,
            "normal_priority_tasks": 0,
            "low_priority_tasks": 0,
            "total_processed": 0
        }
    
    async def start(self):
        """启动系统"""
        self.running = True
        
        self.workers = [
            asyncio.create_task(self._worker(f"priority-worker-{i}"))
            for i in range(self.max_workers)
        ]
        
        print(f"      🚀 优先级生产者消费者系统启动")
    
    async def stop(self):
        """停止系统"""
        self.running = False
        await asyncio.gather(*self.workers, return_exceptions=True)
        print(f"      🛑 优先级生产者消费者系统停止")
    
    async def produce(self, task: TaskItem):
        """生产任务（带优先级）"""
        # 优先级映射：高(0) < 正常(1) < 低(2)
        priority_value = max(0, min(2, task.priority))
        
        try:
            await self.task_queue.put((priority_value, task))
            
            # 更新统计
            if priority_value == 0:
                self.stats["high_priority_tasks"] += 1
            elif priority_value == 1:
                self.stats["normal_priority_tasks"] += 1
            else:
                self.stats["low_priority_tasks"] += 1
            
            return True
        except asyncio.QueueFull:
            return False
    
    async def consume(self) -> Optional[ResultItem]:
        """消费结果"""
        try:
            return await asyncio.wait_for(
                self.result_queue.get(), 
                timeout=1.0
            )
        except asyncio.TimeoutError:
            return None
    
    async def _worker(self, worker_id: str):
        """工作进程"""
        while self.running:
            try:
                # 获取任务（按优先级）
                priority, task = await asyncio.wait_for(
                    self.task_queue.get(), 
                    timeout=1.0
                )
                
                # 处理任务
                start_time = time.time()
                result = await self._process_task(task, worker_id)
                processing_time = time.time() - start_time
                
                # 创建结果项
                result_item = ResultItem(
                    task_id=task.id,
                    result=result,
                    processing_time=processing_time,
                    worker_id=worker_id
                )
                
                await self.result_queue.put(result_item)
                self.stats["total_processed"] += 1
                
                priority_name = ["高", "正常", "低"][priority]
                print(f"      ✅ {worker_id} 完成{priority_name}优先级任务: {task.id}")
                
            except asyncio.TimeoutError:
                continue
            except Exception as e:
                print(f"      ❌ {worker_id} 任务失败: {e}")
    
    async def _process_task(self, task: TaskItem, worker_id: str) -> Any:
        """处理任务"""
        # 高优先级任务处理更快
        if task.priority == 0:
            processing_time = random.uniform(0.05, 0.3)
        elif task.priority == 1:
            processing_time = random.uniform(0.1, 0.5)
        else:
            processing_time = random.uniform(0.2, 0.8)
        
        await asyncio.sleep(processing_time)
        
        return {
            "task_data": task.data,
            "priority": task.priority,
            "processed_by": worker_id,
            "processing_time": processing_time
        }
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        return {
            **self.stats,
            "queue_size": self.task_queue.qsize(),
            "result_queue_size": self.result_queue.qsize()
        }

print(f"   ✅ 生产者消费者模式完成")
print(f"      - AsyncProducerConsumer: 基础异步生产者消费者")
print(f"      - PriorityProducerConsumer: 优先级生产者消费者")

# 2. 工作池模式
print(f"\n   🏊 2. 工作池模式:")

# 2.1 异步工作池
print(f"\n      2.1 异步工作池:")

class AsyncWorkerPool:
    """异步工作池"""
    
    def __init__(self, max_workers: int = 5, max_queue_size: int = 100):
        self.max_workers = max_workers
        self.max_queue_size = max_queue_size
        self.task_queue = asyncio.Queue(maxsize=max_queue_size)
        self.result_futures = {}
        self.workers = []
        self.running = False
        self.worker_stats = {}
    
    async def start(self):
        """启动工作池"""
        self.running = True
        
        # 启动工作进程
        self.workers = [
            asyncio.create_task(self._worker(f"pool-worker-{i}"))
            for i in range(self.max_workers)
        ]
        
        print(f"      🚀 异步工作池启动 ({self.max_workers} 个工作进程)")
    
    async def stop(self):
        """停止工作池"""
        self.running = False
        
        # 取消所有待处理的任务
        for future in self.result_futures.values():
            if not future.done():
                future.cancel()
        
        # 等待工作进程完成
        await asyncio.gather(*self.workers, return_exceptions=True)
        
        print(f"      🛑 异步工作池停止")
    
    async def submit(self, task_func: Callable, *args, **kwargs) -> asyncio.Future:
        """提交任务到工作池"""
        if not self.running:
            raise RuntimeError("工作池未启动")
        
        # 创建结果Future
        result_future = asyncio.Future()
        task_id = str(uuid.uuid4())[:8]
        
        # 存储Future
        self.result_futures[task_id] = result_future
        
        # 提交任务到队列
        task_data = {
            "id": task_id,
            "func": task_func,
            "args": args,
            "kwargs": kwargs
        }
        
        try:
            await self.task_queue.put(task_data)
            return result_future
        except asyncio.QueueFull:
            # 队列满，设置异常并返回
            result_future.set_exception(Exception("任务队列已满"))
            del self.result_futures[task_id]
            return result_future
    
    async def _worker(self, worker_id: str):
        """工作进程"""
        print(f"      👷 {worker_id} 开始工作")
        
        # 初始化工作统计
        self.worker_stats[worker_id] = {
            "tasks_completed": 0,
            "tasks_failed": 0,
            "total_time": 0
        }
        
        while self.running:
            try:
                # 获取任务
                task_data = await asyncio.wait_for(
                    self.task_queue.get(), 
                    timeout=1.0
                )
                
                task_id = task_data["id"]
                start_time = time.time()
                
                try:
                    # 执行任务
                    if asyncio.iscoroutinefunction(task_data["func"]):
                        result = await task_data["func"](*task_data["args"], **task_data["kwargs"])
                    else:
                        # 同步函数在线程池中执行
                        loop = asyncio.get_event_loop()
                        result = await loop.run_in_executor(
                            None, 
                            task_data["func"], 
                            *task_data["args"], 
                            **task_data["kwargs"]
                        )
                    
                    # 设置结果
                    if task_id in self.result_futures:
                        self.result_futures[task_id].set_result(result)
                        del self.result_futures[task_id]
                    
                    # 更新统计
                    processing_time = time.time() - start_time
                    self.worker_stats[worker_id]["tasks_completed"] += 1
                    self.worker_stats[worker_id]["total_time"] += processing_time
                    
                    print(f"      ✅ {worker_id} 完成任务: {task_id} ({processing_time:.3f}s)")
                    
                except Exception as e:
                    # 设置异常
                    if task_id in self.result_futures:
                        self.result_futures[task_id].set_exception(e)
                        del self.result_futures[task_id]
                    
                    # 更新统计
                    self.worker_stats[worker_id]["tasks_failed"] += 1
                    
                    print(f"      ❌ {worker_id} 任务失败: {task_id} - {e}")
                
            except asyncio.TimeoutError:
                continue
            except Exception as e:
                print(f"      ❌ {worker_id} 工作进程错误: {e}")
        
        print(f"      👷 {worker_id} 停止工作")
    
    def get_pool_stats(self) -> Dict[str, Any]:
        """获取工作池统计"""
        total_completed = sum(stats["tasks_completed"] for stats in self.worker_stats.values())
        total_failed = sum(stats["tasks_failed"] for stats in self.worker_stats.values())
        total_time = sum(stats["total_time"] for stats in self.worker_stats.values())
        
        return {
            "max_workers": self.max_workers,
            "active_workers": len(self.workers),
            "queue_size": self.task_queue.qsize(),
            "pending_futures": len(self.result_futures),
            "total_completed": total_completed,
            "total_failed": total_failed,
            "success_rate": f"{total_completed/(total_completed+total_failed)*100:.1f}%" if (total_completed+total_failed) > 0 else "0%",
            "average_time": f"{total_time/total_completed:.3f}s" if total_completed > 0 else "0s",
            "worker_stats": self.worker_stats
        }

# 2.2 线程池工作池
print(f"\n      2.2 线程池工作池:")

class ThreadPoolWorkerPool:
    """线程池工作池（用于CPU密集型任务）"""
    
    def __init__(self, max_workers: int = 4):
        self.max_workers = max_workers
        self.executor = ThreadPoolExecutor(max_workers=max_workers)
        self.loop = None
        self.running = False
        self.task_counter = 0
        self.stats = {
            "submitted_tasks": 0,
            "completed_tasks": 0,
            "failed_tasks": 0
        }
    
    async def start(self):
        """启动线程池工作池"""
        self.running = True
        self.loop = asyncio.get_event_loop()
        print(f"      🚀 线程池工作池启动 ({self.max_workers} 个线程)")
    
    async def stop(self):
        """停止线程池工作池"""
        self.running = False
        self.executor.shutdown(wait=True)
        print(f"      🛑 线程池工作池停止")
    
    async def submit(self, func: Callable, *args, **kwargs) -> asyncio.Future:
        """提交任务到线程池"""
        if not self.running:
            raise RuntimeError("线程池工作池未启动")
        
        self.task_counter += 1
        self.stats["submitted_tasks"] += 1
        
        try:
            # 在线程池中执行任务
            future = await self.loop.run_in_executor(
                self.executor, 
                self._execute_task, 
                func, 
                args, 
                kwargs
            )
            return future
        except Exception as e:
            self.stats["failed_tasks"] += 1
            raise e
    
    def _execute_task(self, func: Callable, args: tuple, kwargs: dict) -> Any:
        """在线程池中执行任务"""
        try:
            result = func(*args, **kwargs)
            self.stats["completed_tasks"] += 1
            return result
        except Exception as e:
            self.stats["failed_tasks"] += 1
            raise e
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        total = self.stats["submitted_tasks"]
        completed = self.stats["completed_tasks"]
        failed = self.stats["failed_tasks"]
        
        return {
            **self.stats,
            "success_rate": f"{completed/total*100:.1f}%" if total > 0 else "0%",
            "pending_tasks": total - completed - failed
        }

print(f"   ✅ 工作池模式完成")
print(f"      - AsyncWorkerPool: 异步工作池")
print(f"      - ThreadPoolWorkerPool: 线程池工作池")

# 3. 扇出扇入模式
print(f"\n   🌊 3. 扇出扇入模式:")

# 3.1 扇出扇入基础实现
print(f"\n      3.1 扇出扇入基础实现:")

class FanOutFanIn:
    """扇出扇入模式实现"""
    
    def __init__(self, max_concurrent: int = 10):
        self.max_concurrent = max_concurrent
        self.semaphore = asyncio.Semaphore(max_concurrent)
        self.stats = {
            "fan_out_tasks": 0,
            "fan_in_results": 0,
            "failed_tasks": 0,
            "total_time": 0
        }
    
    async def fan_out(self, tasks: List[Callable]) -> List[asyncio.Task]:
        """扇出：创建多个并发任务"""
        print(f"      🌊 扇出 {len(tasks)} 个任务")
        
        # 创建并发任务
        concurrent_tasks = []
        
        for i, task_func in enumerate(tasks):
            task = asyncio.create_task(
                self._execute_with_semaphore(task_func, f"task-{i}")
            )
            concurrent_tasks.append(task)
            self.stats["fan_out_tasks"] += 1
        
        return concurrent_tasks
    
    async def fan_in(self, tasks: List[asyncio.Task]) -> List[Any]:
        """扇入：收集并发任务结果"""
        print(f"      🔄 扇入 {len(tasks)} 个任务结果")
        
        start_time = time.time()
        
        # 等待所有任务完成
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # 处理结果
        successful_results = []
        for i, result in enumerate(results):
            if isinstance(result, Exception):
                print(f"      ❌ 任务 {i} 失败: {result}")
                self.stats["failed_tasks"] += 1
            else:
                successful_results.append(result)
                self.stats["fan_in_results"] += 1
        
        total_time = time.time() - start_time
        self.stats["total_time"] += total_time
        
        print(f"      ✅ 扇入完成: {len(successful_results)}/{len(tasks)} 成功 ({total_time:.3f}s)")
        
        return successful_results
    
    async def _execute_with_semaphore(self, task_func: Callable, task_id: str) -> Any:
        """使用信号量控制并发的任务执行"""
        async with self.semaphore:
            try:
                if asyncio.iscoroutinefunction(task_func):
                    return await task_func()
                else:
                    # 同步函数在线程池中执行
                    loop = asyncio.get_event_loop()
                    return await loop.run_in_executor(None, task_func)
            except Exception as e:
                print(f"      ❌ {task_id} 执行失败: {e}")
                raise e
    
    async def process_batch(self, data_items: List[Any], processor: Callable) -> List[Any]:
        """批量处理数据（扇出扇入模式）"""
        print(f"      📦 批量处理 {len(data_items)} 个数据项")
        
        # 创建处理任务
        tasks = [
            lambda item=item: processor(item) 
            for item in data_items
        ]
        
        # 扇出
        concurrent_tasks = await self.fan_out(tasks)
        
        # 扇入
        results = await self.fan_in(concurrent_tasks)
        
        return results
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        stats = self.stats.copy()
        
        if stats["fan_in_results"] > 0:
            stats["average_time"] = stats["total_time"] / stats["fan_in_results"]
        else:
            stats["average_time"] = 0
        
        total_tasks = stats["fan_out_tasks"]
        successful_tasks = stats["fan_in_results"]
        
        if total_tasks > 0:
            stats["success_rate"] = f"{successful_tasks/total_tasks*100:.1f}%"
        else:
            stats["success_rate"] = "0%"
        
        return stats

# 3.2 分阶段扇出扇入
print(f"\n      3.2 分阶段扇出扇入:")

class StagedFanOutFanIn:
    """分阶段扇出扇入模式"""
    
    def __init__(self, stage_workers: List[int] = None):
        if stage_workers is None:
            stage_workers = [5, 3, 2]  # 默认3个阶段
        
        self.stage_workers = stage_workers
        self.stages = []
        self.stats = {
            "stage_stats": [{
                "input_items": 0,
                "output_items": 0,
                "failed_items": 0,
                "processing_time": 0
            } for _ in stage_workers]
        }
    
    async def process_pipeline(self, data_items: List[Any], 
                               processors: List[Callable]) -> List[Any]:
        """多阶段流水线处理"""
        if len(processors) != len(self.stage_workers):
            raise ValueError("处理器数量必须与阶段数量匹配")
        
        print(f"      🏭 启动 {len(processors)} 阶段流水线处理")
        
        current_data = data_items
        
        for stage_idx, (processor, max_workers) in enumerate(zip(processors, self.stage_workers)):
            print(f"      🔄 阶段 {stage_idx + 1}: {len(current_data)} -> ? (最大 {max_workers} 并发)")
            
            start_time = time.time()
            
            # 当前阶段的扇出扇入处理
            stage_fan_out_fan_in = FanOutFanIn(max_concurrent=max_workers)
            
            # 处理当前阶段
            current_data = await stage_fan_out_fan_in.process_batch(
                current_data, 
                processor
            )
            
            processing_time = time.time() - start_time
            
            # 更新阶段统计
            stage_stats = stage_fan_out_fan_in.get_stats()
            self.stats["stage_stats"][stage_idx] = {
                "input_items": stage_stats["fan_out_tasks"],
                "output_items": stage_stats["fan_in_results"],
                "failed_items": stage_stats["failed_tasks"],
                "processing_time": processing_time
            }
            
            print(f"      ✅ 阶段 {stage_idx + 1} 完成: {len(current_data)} 个结果 ({processing_time:.3f}s)")
        
        return current_data
    
    def get_pipeline_stats(self) -> Dict[str, Any]:
        """获取流水线统计"""
        total_input = sum(stage["input_items"] for stage in self.stats["stage_stats"])
        total_output = self.stats["stage_stats"][-1]["output_items"]
        total_time = sum(stage["processing_time"] for stage in self.stats["stage_stats"])
        
        return {
            "total_stages": len(self.stage_workers),
            "stage_workers": self.stage_workers,
            "total_input_items": total_input,
            "total_output_items": total_output,
            "total_processing_time": total_time,
            "overall_throughput": total_output / total_time if total_time > 0 else 0,
            "stage_stats": self.stats["stage_stats"]
        }

print(f"   ✅ 扇出扇入模式完成")
print(f"      - FanOutFanIn: 基础扇出扇入")
print(f"      - StagedFanOutFanIn: 分阶段扇出扇入")

print(f"\n✅ 并发编程模式完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握生产者消费者模式的异步实现")
print(f"   ✓ 学会工作池和线程池的并发管理")
print(f"   ✓ 理解扇出扇入模式的数据处理流程")
print(f"   ✓ 能够应用并发模式解决实际问题")

### 并发模式实战应用 [⭐⭐进阶]
**知识点说明**：并发模式的实战应用是掌握高性能异步编程的关键，对于LangChain的分布式处理和大规模部署具有重要意义。

**学习要求**：
- 掌握并发模式的实际应用场景
- 学会并发性能调优和监控
- 理解并发模式的局限性和适用条件
- 能够设计高效的并发解决方案

**案例要求**：
- 实现完整的并发模式应用系统
- 进行并发性能对比和优化
- 应用并发模式解决LangChain相关问题
- 验证点：能独立设计并实现高性能并发系统

In [ ]:
print("\n🚀 并发模式实战应用:")
print("=" * 50)

# 1. LangChain数据处理并发模式
print(f"📝 1. LangChain数据处理并发模式:")

# 1.1 并发文本处理
print(f"\n   🔍 1.1 并发文本处理:")

@dataclass
class TextProcessingTask:
    """文本处理任务"""
    id: str
    text: str
    operation: str  # 'summarize', 'translate', 'analyze', 'extract'
    priority: int = 1
    metadata: Dict[str, Any] = None

@dataclass
class TextProcessingResult:
    """文本处理结果"""
    task_id: str
    result: str
    operation: str
    processing_time: float
    confidence: float
    worker_id: str

class LangChainTextProcessor:
    """LangChain文本处理器（并发模式）"""
    
    def __init__(self, max_workers: int = 5):
        self.producer_consumer = AsyncProducerConsumer(
            max_queue_size=200, 
            max_workers=max_workers
        )
        self.processing_stats = {
            "summarize": {"count": 0, "total_time": 0},
            "translate": {"count": 0, "total_time": 0},
            "analyze": {"count": 0, "total_time": 0},
            "extract": {"count": 0, "total_time": 0}
        }
    
    async def start(self):
        """启动文本处理器"""
        await self.producer_consumer.start()
        print(f"      🚀 LangChain文本处理器启动")
    
    async def stop(self):
        """停止文本处理器"""
        await self.producer_consumer.stop()
        print(f"      🛑 LangChain文本处理器停止")
    
    async def submit_text_task(self, task: TextProcessingTask) -> str:
        """提交文本处理任务"""
        task_item = TaskItem(
            id=task.id,
            data=task,
            priority=task.priority
        )
        
        success = await self.producer_consumer.produce(task_item)
        if success:
            return task.id
        else:
            raise Exception("任务提交失败")
    
    async def get_result(self) -> Optional[TextProcessingResult]:
        """获取处理结果"""
        result_item = await self.producer_consumer.consume()
        if result_item:
            return result_item.result
        return None
    
    async def _process_task(self, task: TextProcessingTask, worker_id: str) -> TextProcessingResult:
        """处理文本任务"""
        start_time = time.time()
        
        # 模拟不同类型的文本处理
        if task.operation == "summarize":
            processing_time = random.uniform(0.5, 2.0)
            await asyncio.sleep(processing_time)
            
            result = f"Summary of '{task.text[:50]}...': Key points extracted and condensed."
            confidence = random.uniform(0.7, 0.95)
            
        elif task.operation == "translate":
            processing_time = random.uniform(0.3, 1.0)
            await asyncio.sleep(processing_time)
            
            result = f"Translated '{task.text[:30]}...' to target language with high accuracy."
            confidence = random.uniform(0.8, 0.98)
            
        elif task.operation == "analyze":
            processing_time = random.uniform(0.8, 2.5)
            await asyncio.sleep(processing_time)
            
            result = f"Analysis of '{task.text[:40]}...': Sentiment positive, key entities identified."
            confidence = random.uniform(0.6, 0.9)
            
        elif task.operation == "extract":
            processing_time = random.uniform(0.4, 1.5)
            await asyncio.sleep(processing_time)
            
            result = f"Extracted information from '{task.text[:35]}...': Names, dates, locations found."
            confidence = random.uniform(0.75, 0.95)
            
        else:
            raise ValueError(f"不支持的操作: {task.operation}")
        
        # 更新统计
        actual_time = time.time() - start_time
        self.processing_stats[task.operation]["count"] += 1
        self.processing_stats[task.operation]["total_time"] += actual_time
        
        return TextProcessingResult(
            task_id=task.id,
            result=result,
            operation=task.operation,
            processing_time=actual_time,
            confidence=confidence,
            worker_id=worker_id
        )
    
    # 重写父类方法
    async def _process_task_override(self, task: TaskItem, worker_id: str) -> Any:
        """重写任务处理方法"""
        text_task = task.data
        return await self._process_task(text_task, worker_id)
    
    def get_processing_stats(self) -> Dict[str, Any]:
        """获取处理统计"""
        stats = {}
        
        for operation, data in self.processing_stats.items():
            if data["count"] > 0:
                avg_time = data["total_time"] / data["count"]
            else:
                avg_time = 0
            
            stats[operation] = {
                "count": data["count"],
                "total_time": data["total_time"],
                "average_time": avg_time
            }
        
        return {
            "operation_stats": stats,
            "system_stats": self.producer_consumer.get_stats()
        }

# 1.2 并发模型推理
print(f"\n   🤖 1.2 并发模型推理:")

@dataclass
class InferenceTask:
    """推理任务"""
    id: str
    prompt: str
    model: str
    parameters: Dict[str, Any]
    priority: int = 1

@dataclass
class InferenceResult:
    """推理结果"""
    task_id: str
    response: str
    model: str
    tokens_used: int
    processing_time: float
    worker_id: str

class LangChainInferencePool:
    """LangChain推理池（工作池模式）"""
    
    def __init__(self, max_workers: int = 3):
        self.worker_pool = AsyncWorkerPool(
            max_workers=max_workers,
            max_queue_size=100
        )
        self.model_stats = {}
    
    async def start(self):
        """启动推理池"""
        await self.worker_pool.start()
        print(f"      🚀 LangChain推理池启动")
    
    async def stop(self):
        """停止推理池"""
        await self.worker_pool.stop()
        print(f"      🛑 LangChain推理池停止")
    
    async def submit_inference(self, task: InferenceTask) -> asyncio.Future:
        """提交推理任务"""
        return await self.worker_pool.submit(
            self._perform_inference,
            task
        )
    
    async def _perform_inference(self, task: InferenceTask) -> InferenceResult:
        """执行推理任务"""
        start_time = time.time()
        worker_id = f"inference-worker-{random.randint(1, 100)}"
        
        # 模拟不同模型的推理时间
        if task.model == "gpt-4":
            processing_time = random.uniform(1.0, 3.0)
        elif task.model == "gpt-3.5-turbo":
            processing_time = random.uniform(0.5, 1.5)
        elif task.model == "claude-3":
            processing_time = random.uniform(0.8, 2.0)
        else:
            processing_time = random.uniform(0.3, 1.0)
        
        await asyncio.sleep(processing_time)
        
        # 模拟推理结果
        response = f"Model {task.model} response to '{task.prompt[:50]}...': Generated comprehensive answer."
        tokens_used = len(response.split()) + len(task.prompt.split())
        
        actual_time = time.time() - start_time
        
        # 更新模型统计
        if task.model not in self.model_stats:
            self.model_stats[task.model] = {
                "count": 0,
                "total_time": 0,
                "total_tokens": 0
            }
        
        self.model_stats[task.model]["count"] += 1
        self.model_stats[task.model]["total_time"] += actual_time
        self.model_stats[task.model]["total_tokens"] += tokens_used
        
        return InferenceResult(
            task_id=task.id,
            response=response,
            model=task.model,
            tokens_used=tokens_used,
            processing_time=actual_time,
            worker_id=worker_id
        )
    
    def get_inference_stats(self) -> Dict[str, Any]:
        """获取推理统计"""
        stats = {}
        
        for model, data in self.model_stats.items():
            if data["count"] > 0:
                avg_time = data["total_time"] / data["count"]
                avg_tokens = data["total_tokens"] / data["count"]
            else:
                avg_time = avg_tokens = 0
            
            stats[model] = {
                "count": data["count"],
                "average_time": avg_time,
                "average_tokens": avg_tokens,
                "total_tokens": data["total_tokens"]
            }
        
        return {
            "model_stats": stats,
            "pool_stats": self.worker_pool.get_pool_stats()
        }

print(f"   ✅ LangChain数据处理并发模式完成")
print(f"      - LangChainTextProcessor: 并发文本处理")
print(f"      - LangChainInferencePool: 并发模型推理")

# 2. 性能对比和优化
print(f"\n   ⚡ 2. 性能对比和优化:")

async def demonstrate_concurrent_performance():
    """演示并发性能对比"""
    print(f"   🔸 并发性能对比演示:")
    
    # 2.1 生产者消费者 vs 顺序处理
    print(f"\n      2.1 生产者消费者 vs 顺序处理:")
    
    # 创建测试任务
    test_tasks = [
        TaskItem(
            id=f"task-{i}",
            data={"content": f"Test data {i}", "size": i * 10},
            priority=random.randint(0, 2)
        )
        for i in range(20)
    ]
    
    # 顺序处理基准
    print(f"\n        顺序处理基准测试...")
    start_time = time.time()
    
    sequential_results = []
    for task in test_tasks:
        # 模拟顺序处理
        processing_time = random.uniform(0.1, 0.5)
        await asyncio.sleep(processing_time)
        
        result = {
            "task_id": task.id,
            "processed": True,
            "time": processing_time
        }
        sequential_results.append(result)
    
    sequential_time = time.time() - start_time
    print(f"        顺序处理完成: {len(sequential_results)} 个任务 ({sequential_time:.3f}s)")
    
    # 生产者消费者处理
    print(f"\n        生产者消费者处理测试...")
    
    pc_system = AsyncProducerConsumer(max_queue_size=50, max_workers=5)
    await pc_system.start()
    
    start_time = time.time()
    
    # 提交所有任务
    for task in test_tasks:
        await pc_system.produce(task)
    
    # 收集结果
    pc_results = []
    while len(pc_results) < len(test_tasks):
        result = await pc_system.consume()
        if result:
            pc_results.append(result)
    
    pc_time = time.time() - start_time
    pc_stats = pc_system.get_stats()
    
    await pc_system.stop()
    
    print(f"        生产者消费者处理完成: {len(pc_results)} 个任务 ({pc_time:.3f}s)")
    
    # 性能对比
    speedup = sequential_time / pc_time
    print(f"\n        性能对比结果:")
    print(f"          顺序处理时间: {sequential_time:.3f}s")
    print(f"          并发处理时间: {pc_time:.3f}s")
    print(f"          加速比: {speedup:.2f}x")
    print(f"          成功率: {pc_stats['consumed_tasks']}/{pc_stats['produced_tasks']} ({pc_stats['consumed_tasks']/pc_stats['produced_tasks']*100:.1f}%)")
    
    # 2.2 工作池 vs 扇出扇入
    print(f"\n      2.2 工作池 vs 扇出扇入:")
    
    # 创建计算密集型任务
    def cpu_intensive_task(n: int) -> int:
        """CPU密集型任务"""
        result = 0
        for i in range(n):
            result += i * i
        return result
    
    cpu_tasks = [1000, 2000, 1500, 3000, 1200, 2500, 1800, 2200]
    
    # 工作池处理
    print(f"\n        工作池处理测试...")
    
    thread_pool = ThreadPoolWorkerPool(max_workers=4)
    await thread_pool.start()
    
    start_time = time.time()
    
    # 提交CPU密集型任务
    pool_futures = [
        await thread_pool.submit(cpu_intensive_task, n)
        for n in cpu_tasks
    ]
    
    # 等待结果
    pool_results = []
    for future in pool_futures:
        pool_results.append(await future)
    
    pool_time = time.time() - start_time
    pool_stats = thread_pool.get_stats()
    
    await thread_pool.stop()
    
    print(f"        工作池处理完成: {len(pool_results)} 个任务 ({pool_time:.3f}s)")
    
    # 扇出扇入处理
    print(f"\n        扇出扇入处理测试...")
    
    fan_out_fan_in = FanOutFanIn(max_concurrent=4)
    
    start_time = time.time()
    
    # 创建任务函数
    fan_tasks = [
        lambda n=n: cpu_intensive_task(n)
        for n in cpu_tasks
    ]
    
    # 扇出扇入处理
    fan_results = await fan_out_fan_in.process_batch(
        [None] * len(cpu_tasks),  # 占位符
        lambda _, n=n: cpu_intensive_task(n)  # 实际处理函数
    )
    
    fan_time = time.time() - start_time
    fan_stats = fan_out_fan_in.get_stats()
    
    print(f"        扇出扇入处理完成: {len(fan_results)} 个任务 ({fan_time:.3f}s)")
    
    # 性能对比
    print(f"\n        CPU密集型任务性能对比:")
    print(f"          工作池处理时间: {pool_time:.3f}s")
    print(f"          扇出扇入处理时间: {fan_time:.3f}s")
    print(f"          工作池成功率: {pool_stats['success_rate']}")
    print(f"          扇出扇入成功率: {fan_stats['success_rate']}")
    
    return {
        "sequential_vs_concurrent": {
            "sequential_time": sequential_time,
            "concurrent_time": pc_time,
            "speedup": speedup
        },
        "pool_vs_fan": {
            "pool_time": pool_time,
            "fan_time": fan_time
        }
    }

# 运行性能对比演示
performance_comparison = await demonstrate_concurrent_performance()
print(f"\n   🏁 并发性能对比演示完成")

print(f"\n✅ 并发模式实战应用完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握并发模式的实际应用场景")
print(f"   ✓ 学会并发性能调优和监控")
print(f"   ✓ 理解并发模式的局限性和适用条件")
print(f"   ✓ 能够设计高效的并发解决方案")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**6.6 并发编程模式 [⭐⭐进阶]**
- ✅ 掌握生产者消费者模式的异步实现
- ✅ 学会工作池和线程池的并发管理
- ✅ 理解扇出扇入模式的数据处理流程
- ✅ 能够应用并发模式解决实际问题
- ✅ 掌握并发模式的实际应用场景
- ✅ 学会并发性能调优和监控
- ✅ 理解并发模式的局限性和适用条件
- ✅ 能够设计高效的并发解决方案
- ✅ 能独立设计和实现并发解决方案

### 🎯 与LangChain学习的关联

**并发编程模式重要性**：
- 生产者消费者模式优化LangChain的数据处理流程
- 工作池模式提升LangChain的并发推理能力
- 扇出扇入模式增强LangChain的并行处理效率
- 并发模式为LangChain的分布式部署提供基础
- 性能优化技术提升LangChain的大规模处理能力

**实际应用场景**：
- LangChain的并发文本处理和分析
- LangChain的多模型并行推理系统
- LangChain的批量数据处理管道
- LangChain的实时流式处理架构
- LangChain的分布式任务调度系统

### 📚 进阶学习建议

1. **练习建议**：
   - 深入练习复杂并发模式的设计和实现
   - 掌握并发性能调优和瓶颈分析技术
   - 学习分布式并发和微服务架构

2. **扩展学习**：
   - 了解并发编程的理论基础和算法
   - 学习高级并发模式和设计模式
   - 探索并发系统的监控和调试技术

3. **实际应用**：
   - 构建高性能LangChain并发处理系统
   - 开发分布式AI模型推理平台
   - 实现实时大数据处理和分析系统

### 🔧 常见错误与注意事项

1. **并发竞争条件**：
   ```python
   # 错误：没有适当的同步机制
   shared_data = []
   
   async def process_data(item):
       # 多个协程同时修改共享数据
       shared_data.append(item)  # 可能导致数据竞争
   
   # 正确：使用线程安全的数据结构或锁
   import asyncio
   shared_data = []
   data_lock = asyncio.Lock()
   
   async def process_data(item):
       async with data_lock:
           shared_data.append(item)
   
   # 或者使用队列
   from asyncio import Queue
   data_queue = Queue()
   
   async def process_data(item):
       await data_queue.put(item)
   ```

2. **过度并发导致资源耗尽**：
   ```python
   # 错误：无限制并发
   async def unlimited_concurrent(tasks):
       # 可能创建过多协程，耗尽系统资源
       return await asyncio.gather(*[
           asyncio.create_task(process(task))
           for task in tasks
       ])
   
   # 正确：使用信号量控制并发数
   async def limited_concurrent(tasks, max_concurrent=10):
       semaphore = asyncio.Semaphore(max_concurrent)
       
       async def process_with_semaphore(task):
           async with semaphore:
               return await process(task)
       
       return await asyncio.gather(*[
           asyncio.create_task(process_with_semaphore(task))
           for task in tasks
       ])
   ```

3. **阻塞操作在异步环境中**：
   ```python
   # 错误：在异步函数中使用阻塞操作
   async def bad_async_function():
       time.sleep(1)  # 阻塞整个事件循环
       result = some_blocking_io()  # 阻塞I/O操作
       return result
   
   # 正确：使用异步操作或在线程池中执行
   import asyncio
   from concurrent.futures import ThreadPoolExecutor
   
   async def good_async_function():
       await asyncio.sleep(1)  # 异步睡眠
       
       # 在线程池中执行阻塞操作
       loop = asyncio.get_event_loop()
       result = await loop.run_in_executor(None, some_blocking_io)
       return result
   ```

4. **错误处理不当**：
   ```python
   # 错误：忽略并发任务中的异常
   async def process_concurrent_tasks(tasks):
       # 异常被忽略，可能导致静默失败
       await asyncio.gather(*[
           asyncio.create_task(process(task))
           for task in tasks
       ])
   
   # 正确：适当的错误处理
   async def process_concurrent_tasks(tasks):
       results = await asyncio.gather(*[
           asyncio.create_task(process(task))
           for task in tasks
       ], return_exceptions=True)
       
       successful_results = []
       errors = []
       
       for i, result in enumerate(results):
           if isinstance(result, Exception):
               errors.append((i, result))
               print(f"任务 {i} 失败: {result}")
           else:
               successful_results.append(result)
       
       return successful_results, errors
   ```

5. **内存泄漏风险**：
   ```python
   # 错误：任务队列无限增长
   class BadProducerConsumer:
       def __init__(self):
           self.task_queue = asyncio.Queue()  # 无大小限制
       
       async def produce(self, task):
           await self.task_queue.put(task)  # 可能无限增长
   
   # 正确：限制队列大小并处理满队列情况
   class GoodProducerConsumer:
       def __init__(self, max_queue_size=100):
           self.task_queue = asyncio.Queue(maxsize=max_queue_size)
       
       async def produce(self, task):
           try:
               await self.task_queue.put(task)
               return True
           except asyncio.QueueFull:
               print("队列已满，丢弃任务")
               return False
   ```

6. **不公平的任务调度**：
   ```python
   # 错误：长任务阻塞短任务
   async def unfair_scheduler():
       while True:
           task = await queue.get()  # FIFO，长任务可能阻塞短任务
           await process(task)  # 长任务会占用工作进程
   
   # 正确：使用优先级队列或任务分片
   async def fair_scheduler():
       priority_queue = asyncio.PriorityQueue()
       
       while True:
           priority, task = await priority_queue.get()
           
           # 为长任务设置较低优先级
           if is_long_task(task):
               priority = 2
           else:
               priority = 0
           
           await priority_queue.put((priority, task))
   ```

### 🌐 性能优化建议

**并发模式优化**：
- 根据任务类型选择合适的并发模式
- 合理设置工作进程数量和队列大小
- 使用优先级队列优化任务调度
- 实现动态负载均衡和资源分配

**系统优化**：
- 监控系统资源使用情况
- 实现优雅降级和故障恢复
- 使用缓存减少重复计算
- 优化数据结构和算法复杂度

**监控和调试**：
- 实现详细的性能监控和日志
- 使用分析工具识别性能瓶颈
- 建立告警机制和自动扩展
- 定期进行性能测试和优化

---

**🎉 恭喜完成并发编程模式学习！**

你已经全面掌握了并发编程模式的核心技术，能够系统性地进行生产者消费者、工作池、扇出扇入等模式的设计和实现，为LangChain智能应用提供了强大的并发处理基础。

## 🚀 下一步学习预告

**第六节：异步编程 - 进行中** 🔄
- 6.1 异步编程基础 ✅
- 6.2 协程与事件循环 ✅
- 6.3 异步I/O操作 ✅
- 6.4 异步网络编程 ✅
- 6.5 异步Web框架 ✅
- 6.6 并发编程模式 ✅
- 6.7 异步任务队列 ⏳
- 6.8 异步性能优化 ⏳
- 6.9 异步最佳实践 ⏳

**继续第六节：异步编程**
- 下一个：6.7 异步任务队列
- 深入学习任务队列和分布式处理
- 掌握Celery类似的异步任务系统

**后续章节预告**：
- Web开发技术
- 项目工程实践

继续加油，并发编程模式已经掌握！准备深入学习异步任务队列！